In [ ]:
!pip install dash-extensions

In [1]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px
from dash_extensions.enrich import Output, DashProxy, Input, MultiplexerTransform ,State

C:\Users\xy\AppData\Local\Temp\ipykernel_16312\3314491980.py:4: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
C:\Users\xy\AppData\Local\Temp\ipykernel_16312\3314491980.py:5: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc


In [2]:
!wget "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"

--2023-04-24 17:49:37--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 198.23.119.245
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|198.23.119.245|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2476 (2.4K) [text/csv]
Saving to: 'spacex_launch_dash.csv.4'

     0K ..                                                    100%  621M=0s

2023-04-24 17:49:39 (621 MB/s) - 'spacex_launch_dash.csv.4' saved [2476/2476]



In [3]:
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

In [51]:
def pie(site_dropdown):
    if site_dropdown=='ALL':
        all_site = spacex_df[['Launch Site','class']].groupby('Launch Site').mean().reset_index()
        fig1 = px.pie(all_site, values='class', names='Launch Site',hole=0.3, title='Total Success Launches for all sites')
        return fig1
    else:
        one_site=spacex_df[['Launch Site','class']]
        specific_site = one_site[one_site['Launch Site']==site_dropdown].value_counts().to_frame().reset_index()
        specific_site['count']=specific_site[0]
        specific_site.drop(0,axis=1,inplace=True)
        specific_site
        fig1 = px.pie(specific_site, values='count', names='class',hole=0.3, title='Total Success Launches for Site '+site_dropdown)
        return fig1

In [52]:
pie('ALL')

In [31]:
def scatter(site_dropdown,payload_slider):
    if site_dropdown == 'ALL':
        mass_range = spacex_df[(spacex_df['Payload Mass (kg)']>=payload_slider[0]) & (spacex_df['Payload Mass (kg)']<=payload_slider[1])]
        data = mass_range[['Payload Mass (kg)','class','Booster Version Category','Launch Site']]
        fig2 = px.scatter(data,x='Payload Mass (kg)',y='class',color='Booster Version Category',title='Correlation between Payload and Success for all sites' )
        fig2.update_traces(marker_size=30, marker_line=dict(width=2))
        return fig2
    else:
        mass_range = spacex_df[(spacex_df['Payload Mass (kg)']>=payload_slider[0]) & (spacex_df['Payload Mass (kg)']<=payload_slider[1])]
        data = mass_range[['Payload Mass (kg)','class','Booster Version Category','Launch Site']]
        data = data[data['Launch Site']==site_dropdown]
        fig2 = px.scatter(data,x='Payload Mass (kg)',y='class',color='Booster Version Category',title='Correlation between Payload and Success for '+site_dropdown)
        fig2.update_traces(marker_size=30, marker_line=dict(width=2))
        return fig2

In [30]:
scatter('ALL',[5000,10000])

In [53]:
# Create a dash application
app = dash.Dash(__name__)
#app = DashProxy(prevent_initial_callbacks=True, transforms=[MultiplexerTransform()])
# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(id='site-dropdown',
                                options=[
                                {'label': 'All Sites', 'value': 'ALL'},
                                {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},    
                                {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},                                  
                                ],
                                value='ALL',
                                placeholder="Select a Launch Site here",
                                searchable=True
                                ),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',
                                min=0,max=10000,step=1000,
                                value=[min_payload,max_payload],
                                marks={0: '0', 2500:'2500',5000:'5000',
                                7500:'7500', 10000: '10000'}),
                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def pie(site_dropdown):
    if site_dropdown=='ALL':
        all_site = spacex_df[['Launch Site','class']].groupby('Launch Site').mean().reset_index()
        fig1 = px.pie(all_site, values='class', names='Launch Site',hole=0.3, title='Total Success Launches for all sites')
        return fig1
    else:
        one_site=spacex_df[['Launch Site','class']]
        specific_site = one_site[one_site['Launch Site']==site_dropdown].value_counts().to_frame().reset_index()
        specific_site['count']=specific_site[0]
        specific_site.drop(0,axis=1,inplace=True)
        specific_site
        fig1 = px.pie(specific_site, values='count', names='class',hole=0.3, title='Total Success Launches for Site '+site_dropdown)
        return fig1

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              [Input(component_id='site-dropdown', component_property='value'),
              Input(component_id='payload-slider', component_property='value')])
def scatter(site_dropdown,payload_slider):
    if site_dropdown == 'ALL':
        mass_range = spacex_df[(spacex_df['Payload Mass (kg)']>=payload_slider[0]) & (spacex_df['Payload Mass (kg)']<=payload_slider[1])]
        data = mass_range[['Payload Mass (kg)','class','Booster Version Category','Launch Site']]
        fig2 = px.scatter(data,x='Payload Mass (kg)',y='class',color='Booster Version Category',title='Correlation between Payload and Success for all sites' )
        fig2.update_traces(marker_size=30, marker_line=dict(width=2))
        return fig2
    else:
        mass_range = spacex_df[(spacex_df['Payload Mass (kg)']>=payload_slider[0]) & (spacex_df['Payload Mass (kg)']<=payload_slider[1])]
        data = mass_range[['Payload Mass (kg)','class','Booster Version Category','Launch Site']]
        data = data[data['Launch Site']==site_dropdown]
        fig2 = px.scatter(data,x='Payload Mass (kg)',y='class',color='Booster Version Category',title='Correlation between Payload and Success for '+site_dropdown)
        fig2.update_traces(marker_size=30, marker_line=dict(width=2))
        return fig2



In [54]:
# Run the app
if __name__ == '__main__':
    app.run_server(port=6060)

Dash is running on http://127.0.0.1:6060/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:6060/ (Press CTRL+C to quit)
127.0.0.1 - - [24/Apr/2023 18:09:00] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [24/Apr/2023 18:09:01] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [24/Apr/2023 18:09:01] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [24/Apr/2023 18:09:01] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [24/Apr/2023 18:09:01] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [24/Apr/2023 18:09:01] "GET /_dash-component-suites/dash/dcc/async-slider.js HTTP/1.1" 200 -
127.0.0.1 - - [24/Apr/2023 18:09:01] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [24/Apr/2023 18:09:01] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [24/Apr/2023 18:09:01] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [24/Apr/2023 18:12:40] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [24/Apr/2023 18:12:40] "POST /_dash-

127.0.0.1 - - [24/Apr/2023 19:02:15] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [24/Apr/2023 19:02:17] "POST /_dash-update-component HTTP/1.1" 200 -
